In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

driver = webdriver.Chrome()
url = "https://taas.koroad.or.kr/sta/acs/exs/typical.do?menuId=WEB_KMP_OVT_UAS_ASA"
driver.get(url)

In [2]:
# 좌측 교통사고 (경찰DB) 클릭 
driver.find_element(By.CSS_SELECTOR,"#jdMenuTree17").click()

In [3]:
# 사고일반
driver.find_element(By.CSS_SELECTOR,"#ddMenuTree17 > div:nth-child(5) > a.node").click()

In [4]:
import time 
# 사고유형별 시간대별 교통사고 클릭 
driver.find_element(By.CSS_SELECTOR,"#sdMenuTree32").click()

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

index = 0
for n in range(10,20):
    year = 2004 + n
    driver = webdriver.Chrome()
    url = "https://taas.koroad.or.kr/sta/acs/exs/typical.do?menuId=WEB_KMP_OVT_UAS_ASA"
    driver.get(url)
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR,"#jdMenuTree17").click()
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR,"#ddMenuTree17 > div:nth-child(5) > a.node").click()
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR,"#sdMenuTree32").click()
    time.sleep(3)
    # 년도 시작
    driver.find_element(By.CSS_SELECTOR,f"#startYear > option:nth-child({n})").click()
    time.sleep(3)
    # 년도 끝
    driver.find_element(By.CSS_SELECTOR,f"#endYear > option:nth-child({n})").click()
    time.sleep(3)
    # 조회 
    driver.find_element(By.CSS_SELECTOR,"#searchDiv > ul.top01-03 > li:nth-child(2) > input").click()
    time.sleep(10)
    # iframe 요소 찾기
    iframes = driver.find_elements(By.TAG_NAME,"iframe")
    # iframe 으로 전환 
    driver.switch_to.frame('eosFrame')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    table_rows = soup.select("#OctagonGrid > tbody > tr")
    table_info = []
    curr_topcat = ""
    curr_midcat = ""

    for row in table_rows:
        cells = row.find_all("td")
        texts = [cell.text.strip() for cell in cells]

        if not texts:
            continue

        if len(cells) == 3:
            curr_topcat = texts[0]
            curr_midcat = texts[1]
            구분 = texts[2]
        
        elif len(cells) == 2:
            curr_midcat = texts[0]
            구분 = texts[1]

        elif len(cells) == 1:
            구분 = texts[0]
        
        else:
            continue

        table_info.append((curr_topcat,curr_midcat,구분))

    data_rows = soup.select("#dataSession > tbody > tr")

    시간대_headers = ['00~02시', '02~04시', '04~06시', '06~08시', '08~10시',
                 '10~12시', '12~14시', '14~16시', '16~18시', '18~20시',
                 '20~22시', '22~24시']
    data = []
    
    for idx, row in enumerate(data_rows):
        cells = row.find_all("td")
        values = [cell.text.strip().replace(',','') for cell in cells]
        values = [value.replace('-','0') for value in values]                      # 문자열로 0 처리
        values = [int(value) if value.isdigit() else value for value in values]    # 숫자변환 
        
        if idx >= len(table_info):
            continue

        대분류, 중분류, 구분 = table_info[idx]

        if len(values) != len(시간대_headers):
            print(f"값 개수 불일치 : {values}")
            continue

        
        for 시간대, 값 in zip(시간대_headers,values):
            data.append({
                "id" : index,
                "year_type_id":year,
                "사고유형대분류": 대분류,
                "사고유형중분류": 중분류,
                "구분": 구분,
                "시간대":시간대,
                "값": 값
            })
            index +=1

    df = pd.DataFrame(data)
    df.to_csv(f"{year}년_시간대별_사고통계.csv",index=False, encoding='utf-8-sig')
    print(f"{year}년_저장완료")
    driver.quit()
    time.sleep(3)

2014년_저장완료
2015년_저장완료
2016년_저장완료
2017년_저장완료
2018년_저장완료
2019년_저장완료
2020년_저장완료
2021년_저장완료
2022년_저장완료
2023년_저장완료
